In [27]:
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import optuna
import torch
from optuna.trial import TrialState
from torch.utils.data import Dataset, DataLoader, TensorDataset,Subset
from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
from sklearn.linear_model import LinearRegression
import seaborn as sns
plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

from Dataset import SimulatedSequenceDataset
from LSTMRegressor import LSTMRegressor,save_model,load_model
from LSTMRegressor import DEVICE
from LSTMRegressor import train_model
from LSTMRegressor import test_model
from LSTMRegressor import predict
from LSTMRegressor import plot_train_val_loss_cv
from LSTMRegressor import save_best_trial
from LSTMRegressor import load_best_trial
from LSTMRegressor import Objective_CV,Objective
from metrics import NSE,NNSE,RMSE,NRMSE,ZNRMSE,MAE,PPTS,LPTS,MAPE,R2
from sklearn.metrics import r2_score

hydro_stations = [
    'Tangnaihai',
    'Guide',
    'Xunhua'
]

hydrostation_abbrs = {'Tangnaihai':'TNH','Guide':'GD','Xunhua':'XH'}
hydrostation_channel = {'Tangnaihai':'3','Guide':'14','Xunhua':'9'}
# read observed climate data
hydrostation_metestations = {
    'Tangnaihai': [
        '玛多', '达日', '久治', '红原', '若尔盖', '玛曲', '玛沁', '河南', '兴海',
    ],
    'Guide': [
        '玛多', '达日', '久治', '红原', '若尔盖', '玛曲', '玛沁', '河南', '兴海',
        '贵南', '共和', '贵德',
    ],
    'Xunhua': [
        '玛多', '达日', '久治', '红原', '若尔盖', '玛曲', '玛沁', '河南', '兴海',
        '贵南', '共和', '贵德', '同仁',
    ]
}

In [28]:
selected_features = ['snomlt(mm)', 'eplant(mm)', 'surq_cha(mm)', 
                         'snofall(mm)', 'wet_oflo(mm)', 'wet_evap(mm)', 
                         'snopack(mm)', 'sw_change(mm)', 'MIN-TEM(C)','SWATPlusSimFlow','flow(m^3/s)']
start_year = 1972
pred_years = np.arange(2015,2020)
RETRAIN = True
name = 'InputOutputSamples_hydrosimyr_swatpsim'
batch_size = 32
ystar_col = "pred"
if not os.path.exists(sample_path):
    os.makedirs(sample_path)
for hydro_station in hydro_stations:
    sample_path = f'../samples/SequenceSamples/{hydro_station}/{name}/'
    if not os.path.exists(sample_path):
        os.makedirs(sample_path)
    result_path = f'../result/LSTMRegressorData/{hydro_station}/{name}/'
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    pred_df = pd.DataFrame()
    for pred_year in pred_years:
        model_path = f'../model/12MonthlyFlowProcessForecast_{hydro_station}/LSTMRegressorModel/{name}/pred{pred_year}/'
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        df = pd.read_csv(
            f'../samples/{name}/{hydro_station}_MeteAVGCalvalFeatureDataForML_PRED{pred_year}.csv',
            index_col=['date'], parse_dates=['date']
        )

        df = df[selected_features]

        # Check if 'ARIMASimFlow' exists in df columns
        if 'ARIMASimFlow' not in df.columns:
            print(f"Warning: ARIMASimFlow column not found in data for {hydro_station} station, year {pred_year}")
        
        target = 'flow(m^3/s)'
        features = list(df.columns.difference([target]))
        features = list(df.columns.copy())
        features.remove(target)
        cal = df.loc[:f'{pred_year-6}-12-31',:]
        test = df.loc[f'{pred_year-6}-01-01':f'{pred_year-1}-12-31',:]
        pre = df.loc[f'{pred_year-1}-01-01':f'{pred_year}-12-31',:]
        X_scaler = MinMaxScaler(feature_range=(0,1))
        Y_scaler = MinMaxScaler(feature_range=(0,1))
        X_scaler.fit(cal[features])
        Y_scaler.fit(cal[[target]])
        cal_X = X_scaler.transform(cal[features])
        test_X = X_scaler.transform(test[features])
        pre_X = X_scaler.transform(pre[features])
        cal_y = Y_scaler.transform(cal[[target]])
        test_y = Y_scaler.transform(test[[target]])
        pre_y = Y_scaler.transform(pre[[target]])
        cal = pd.concat([pd.DataFrame(cal_X,columns=features,index=cal.index),pd.DataFrame(cal_y,columns=[target],index=cal.index)],axis=1)
        test = pd.concat([pd.DataFrame(test_X,columns=features,index=test.index),pd.DataFrame(test_y,columns=[target],index=test.index)],axis=1)
        pre = pd.concat([pd.DataFrame(pre_X,columns=features,index=pre.index),pd.DataFrame(pre_y,columns=[target],index=pre.index)],axis=1)
        cal.to_csv(sample_path+f'cal_{start_year}_{pred_year-6}.csv')
        test.to_csv(sample_path+f'test_{pred_year-5}_{pred_year-1}.csv')
        pre.to_csv(sample_path+f'pre_{pred_year}.csv')

        sequence_length =12
        cal_dataset = SimulatedSequenceDataset(
            dataframe=cal.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        test_dataset = SimulatedSequenceDataset(
            dataframe=test.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )

        i=0
        X, y = cal_dataset[i]
        # print(X)
        # print(y)

        # print(cal_dataset.X.shape, cal_dataset.y.shape)
        # print(test_dataset.X.shape, test_dataset.y.shape)

        cal_index = cal.index[sequence_length:]
        test_index = test.index[sequence_length:]

        # print(len(cal_index)==cal_dataset.__len__())
        # print(len(test_index)==test_dataset.__len__())

       
        
        if not os.path.exists(model_path+'model.pickle'):
            objective = Objective(
                    train_dataset=cal_dataset,
                    val_dataset=test_dataset,
                    num_epoch=1000,# 1000
                    batch_size=batch_size,
                    shuffle=True,
                    model_path='./scheme/LSTMRegressor/',
            )
            study = optuna.create_study(
                    study_name='example-study',
                    direction='minimize',
                )
            study.optimize(objective, n_trials=100) #100

            pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
            complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])


            print("Study statistics: ")
            print("  Number of finished trials: ", len(study.trials))
            print("  Number of pruned trials: ", len(pruned_trials))
            print("  Number of complete trials: ", len(complete_trials))

            print("Best trial:")
            trial = study.best_trial

            save_best_trial(trial, model_path=model_path)

            save_model(trial, model_path=model_path)

            best_model_state = trial.user_attrs["best_model_state"]
            print("  Value: ", trial.value)
            print("  Params: ")
            for key, value in trial.params.items():
                print("    {}: {}".format(key, value))

            best_trial = load_best_trial(model_file=model_path+'best_trial.pickle')
            model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
            cal_loader = DataLoader(cal_dataset, batch_size=64, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

            cal_ = cal.loc[cal_index,[target]].copy()
            test_ = test.loc[test_index,[target]].copy()

            cal_[ystar_col] = predict(cal_loader, model).cpu().numpy()
            test_[ystar_col] = predict(test_loader, model).cpu().numpy()

            df_out = pd.concat((cal_, test_))[[target, ystar_col]]
            for c in df_out.columns:
                # df_out[c] = df_out[c] * target_std + target_mean
                df_out[c] = Y_scaler.inverse_transform(df_out[[c]])
            # print(df_out)
            df_out = df_out.rename(columns={target: 'obs'})
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            df_out.to_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv')
        else:
            df_out = pd.read_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv',index_col=['date'], parse_dates=['date'])
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            print(f'model already exists in {model_path}')

        # !Perform prediction
        pre = pd.read_csv(sample_path+f'pre_{pred_year}.csv',index_col=['date'], parse_dates=['date'])
        pre_dataset = SimulatedSequenceDataset(
            dataframe=pre.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        pre_loader = DataLoader(pre_dataset, batch_size=batch_size, shuffle=False)
        pre_index = pre.index[sequence_length:]
        pre_ = pre.loc[pre_index,[target]].copy()
        model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
        pre_[ystar_col] = predict(pre_loader, model).cpu().numpy()
        for c in pre_.columns:
            # df_out[c] = df_out[c] * target_std + target_mean
            pre_[c] = Y_scaler.inverse_transform(pre_[[c]])
        pred_df = pd.concat((pred_df, pre_), axis=0)
    pred_df = pred_df.rename(columns={'flow(m^3/s)': 'obs'})
    print(f'R2 for pred at {hydro_station}({pred_years[0]}-{pred_years[-1]})',r2_score(pred_df['obs'],pred_df['pred']))
    pred_df.to_csv(result_path+f'vif_lstm_pred_{pred_years[0]}_{pred_years[-1]}.csv')
    # Visualization of the flow(m^3/s) and VIF_LSTM of pred_df
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['obs'], mode='lines', name='obs'))
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['pred'], mode='lines', name='pred'))
    fig.show()


R2 for sim at Tangnaihai(1972-2014) 0.9742207343031564
model already exists in ../model/12MonthlyFlowProcessForecast_Tangnaihai/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2015/
R2 for sim at Tangnaihai(1972-2015) 0.8422679737404372
model already exists in ../model/12MonthlyFlowProcessForecast_Tangnaihai/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2016/
R2 for sim at Tangnaihai(1972-2016) 0.905596291249159
model already exists in ../model/12MonthlyFlowProcessForecast_Tangnaihai/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2017/
R2 for sim at Tangnaihai(1972-2017) 0.8815012237392648
model already exists in ../model/12MonthlyFlowProcessForecast_Tangnaihai/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2018/
R2 for sim at Tangnaihai(1972-2018) 0.7722030022279798
model already exists in ../model/12MonthlyFlowProcessForecast_Tangnaihai/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2019/
R2 for pred at Tangna

R2 for sim at Guide(1972-2014) 0.9698826027677683
model already exists in ../model/12MonthlyFlowProcessForecast_Guide/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2015/
R2 for sim at Guide(1972-2015) 0.9615940315245716
model already exists in ../model/12MonthlyFlowProcessForecast_Guide/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2016/
R2 for sim at Guide(1972-2016) 0.9727552161495243
model already exists in ../model/12MonthlyFlowProcessForecast_Guide/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2017/
R2 for sim at Guide(1972-2017) 0.920570431053021
model already exists in ../model/12MonthlyFlowProcessForecast_Guide/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2018/
R2 for sim at Guide(1972-2018) 0.9503388434880898
model already exists in ../model/12MonthlyFlowProcessForecast_Guide/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2019/
R2 for pred at Guide(2015-2019) 0.2774600346584769


R2 for sim at Xunhua(1972-2014) 0.8624612712345638
model already exists in ../model/12MonthlyFlowProcessForecast_Xunhua/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2015/
R2 for sim at Xunhua(1972-2015) 0.9744088740046443
model already exists in ../model/12MonthlyFlowProcessForecast_Xunhua/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2016/
R2 for sim at Xunhua(1972-2016) 0.949707611282172
model already exists in ../model/12MonthlyFlowProcessForecast_Xunhua/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2017/
R2 for sim at Xunhua(1972-2017) 0.7581770111156885
model already exists in ../model/12MonthlyFlowProcessForecast_Xunhua/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2018/
R2 for sim at Xunhua(1972-2018) 0.940988273148866
model already exists in ../model/12MonthlyFlowProcessForecast_Xunhua/LSTMRegressorModel/InputOutputSamples_hydrosimyr_swatpsim/pred2019/
R2 for pred at Xunhua(2015-2019) 0.1673283651923776


In [29]:
selected_features = ['P2020(mm)','MAX-TEM(C)','MIN-TEM(C)','flow(m^3/s)']
start_year = 1972
pred_years = np.arange(2015,2020)
RETRAIN = True
name = 'InputOutputSamples_hydrosimyr'
batch_size = 32
ystar_col = "pred"
if not os.path.exists(sample_path):
    os.makedirs(sample_path)
for hydro_station in hydro_stations:
    sample_path = f'../samples/SequenceSamples/{hydro_station}/{name}/'
    if not os.path.exists(sample_path):
        os.makedirs(sample_path)
    result_path = f'../result/LSTMRegressorData/{hydro_station}/{name}/'
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    pred_df = pd.DataFrame()
    for pred_year in pred_years:
        model_path = f'../model/12MonthlyFlowProcessForecast_{hydro_station}/LSTMRegressorModel/{name}/pred{pred_year}/'
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        df = pd.read_csv(
            f'../samples/{name}/{hydro_station}_MeteAVGCalvalFeatureDataForML_PRED{pred_year}.csv',
            index_col=['date'], parse_dates=['date']
        )

        df = df[selected_features]

        # Check if 'ARIMASimFlow' exists in df columns
        if 'ARIMASimFlow' not in df.columns:
            print(f"Warning: ARIMASimFlow column not found in data for {hydro_station} station, year {pred_year}")
        
        target = 'flow(m^3/s)'
        features = list(df.columns.difference([target]))
        features = list(df.columns.copy())
        features.remove(target)
        cal = df.loc[:f'{pred_year-6}-12-31',:]
        test = df.loc[f'{pred_year-6}-01-01':f'{pred_year-1}-12-31',:]
        pre = df.loc[f'{pred_year-1}-01-01':f'{pred_year}-12-31',:]
        X_scaler = MinMaxScaler(feature_range=(0,1))
        Y_scaler = MinMaxScaler(feature_range=(0,1))
        X_scaler.fit(cal[features])
        Y_scaler.fit(cal[[target]])
        cal_X = X_scaler.transform(cal[features])
        test_X = X_scaler.transform(test[features])
        pre_X = X_scaler.transform(pre[features])
        cal_y = Y_scaler.transform(cal[[target]])
        test_y = Y_scaler.transform(test[[target]])
        pre_y = Y_scaler.transform(pre[[target]])
        cal = pd.concat([pd.DataFrame(cal_X,columns=features,index=cal.index),pd.DataFrame(cal_y,columns=[target],index=cal.index)],axis=1)
        test = pd.concat([pd.DataFrame(test_X,columns=features,index=test.index),pd.DataFrame(test_y,columns=[target],index=test.index)],axis=1)
        pre = pd.concat([pd.DataFrame(pre_X,columns=features,index=pre.index),pd.DataFrame(pre_y,columns=[target],index=pre.index)],axis=1)
        cal.to_csv(sample_path+f'cal_{start_year}_{pred_year-6}.csv')
        test.to_csv(sample_path+f'test_{pred_year-5}_{pred_year-1}.csv')
        pre.to_csv(sample_path+f'pre_{pred_year}.csv')

        sequence_length =12
        cal_dataset = SimulatedSequenceDataset(
            dataframe=cal.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        test_dataset = SimulatedSequenceDataset(
            dataframe=test.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )

        i=0
        X, y = cal_dataset[i]
        # print(X)
        # print(y)

        # print(cal_dataset.X.shape, cal_dataset.y.shape)
        # print(test_dataset.X.shape, test_dataset.y.shape)

        cal_index = cal.index[sequence_length:]
        test_index = test.index[sequence_length:]

        # print(len(cal_index)==cal_dataset.__len__())
        # print(len(test_index)==test_dataset.__len__())

       
        
        if not os.path.exists(model_path+'model.pickle'):
            objective = Objective(
                    train_dataset=cal_dataset,
                    val_dataset=test_dataset,
                    num_epoch=1000,# 1000
                    batch_size=batch_size,
                    shuffle=True,
                    model_path='./scheme/LSTMRegressor/',
            )
            study = optuna.create_study(
                    study_name='example-study',
                    direction='minimize',
                )
            study.optimize(objective, n_trials=100) #100

            pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
            complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])


            print("Study statistics: ")
            print("  Number of finished trials: ", len(study.trials))
            print("  Number of pruned trials: ", len(pruned_trials))
            print("  Number of complete trials: ", len(complete_trials))

            print("Best trial:")
            trial = study.best_trial

            save_best_trial(trial, model_path=model_path)

            save_model(trial, model_path=model_path)

            best_model_state = trial.user_attrs["best_model_state"]
            print("  Value: ", trial.value)
            print("  Params: ")
            for key, value in trial.params.items():
                print("    {}: {}".format(key, value))

            best_trial = load_best_trial(model_file=model_path+'best_trial.pickle')
            model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
            cal_loader = DataLoader(cal_dataset, batch_size=64, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

            cal_ = cal.loc[cal_index,[target]].copy()
            test_ = test.loc[test_index,[target]].copy()

            cal_[ystar_col] = predict(cal_loader, model).cpu().numpy()
            test_[ystar_col] = predict(test_loader, model).cpu().numpy()

            df_out = pd.concat((cal_, test_))[[target, ystar_col]]
            for c in df_out.columns:
                # df_out[c] = df_out[c] * target_std + target_mean
                df_out[c] = Y_scaler.inverse_transform(df_out[[c]])
            # print(df_out)
            df_out = df_out.rename(columns={target: 'obs'})
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            df_out.to_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv')
        else:
            df_out = pd.read_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv',index_col=['date'], parse_dates=['date'])
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            print(f'model already exists in {model_path}')

        # !Perform prediction
        pre = pd.read_csv(sample_path+f'pre_{pred_year}.csv',index_col=['date'], parse_dates=['date'])
        pre_dataset = SimulatedSequenceDataset(
            dataframe=pre.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        pre_loader = DataLoader(pre_dataset, batch_size=batch_size, shuffle=False)
        pre_index = pre.index[sequence_length:]
        pre_ = pre.loc[pre_index,[target]].copy()
        model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
        pre_[ystar_col] = predict(pre_loader, model).cpu().numpy()
        for c in pre_.columns:
            # df_out[c] = df_out[c] * target_std + target_mean
            pre_[c] = Y_scaler.inverse_transform(pre_[[c]])
        pred_df = pd.concat((pred_df, pre_), axis=0)
    pred_df = pred_df.rename(columns={'flow(m^3/s)': 'obs'})
    print(f'R2 for pred at {hydro_station}({pred_years[0]}-{pred_years[-1]})',r2_score(pred_df['obs'],pred_df['pred']))
    pred_df.to_csv(result_path+f'vif_lstm_pred_{pred_years[0]}_{pred_years[-1]}.csv')
    # Visualization of the flow(m^3/s) and VIF_LSTM of pred_df
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['obs'], mode='lines', name='obs'))
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['pred'], mode='lines', name='pred'))
    fig.show()


[I 2024-10-29 12:14:41,249] A new study created in memory with name: example-study


[I 2024-10-29 12:15:35,343] Trial 0 finished with value: 0.002868501760531217 and parameters: {'num_layers': 3, 'hidden_size': 104, 'lstm_dropout': 0.24686982687638526, 'dense_dropout': 0.09843667407880913, 'learning_rate': 0.0010948524590535516}. Best is trial 0 with value: 0.002868501760531217.
[I 2024-10-29 12:16:24,158] Trial 1 finished with value: 0.007857411168515682 and parameters: {'num_layers': 2, 'hidden_size': 64, 'lstm_dropout': 0.14712357590201003, 'dense_dropout': 0.3181419695115645, 'learning_rate': 9.269539776984173e-05}. Best is trial 0 with value: 0.002868501760531217.
[I 2024-10-29 12:17:17,536] Trial 2 finished with value: 0.008487807121127844 and parameters: {'num_layers': 3, 'hidden_size': 88, 'lstm_dropout': 0.11508695043578998, 'dense_dropout': 0.29250456799692076, 'learning_rate': 1.607126384945447e-05}. Best is trial 0 with value: 0.002868501760531217.
[I 2024-10-29 12:18:09,844] Trial 3 finished with value: 0.0062603752594441175 and parameters: {'num_layers':

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  67
  Number of complete trials:  33
Best trial:
  Value:  0.001781786122592166
  Params: 
    num_layers: 1
    hidden_size: 96
    lstm_dropout: 0.034200067707317046
    dense_dropout: 0.06060369010853425
    learning_rate: 0.01610251074565405
R2 for sim at Tangnaihai(1972-2014) 0.9765327917124771


[I 2024-10-29 12:42:35,563] Trial 0 finished with value: 0.005818087141960859 and parameters: {'num_layers': 1, 'hidden_size': 104, 'lstm_dropout': 0.3929800216188265, 'dense_dropout': 0.2251619400195643, 'learning_rate': 1.4352688140048598e-05}. Best is trial 0 with value: 0.005818087141960859.
[I 2024-10-29 12:43:29,960] Trial 1 finished with value: 0.009717746637761593 and parameters: {'num_layers': 3, 'hidden_size': 56, 'lstm_dropout': 0.07227708039040975, 'dense_dropout': 0.4036788718346034, 'learning_rate': 0.029721163083579628}. Best is trial 0 with value: 0.005818087141960859.
[I 2024-10-29 12:44:16,631] Trial 2 finished with value: 0.010389550821855664 and parameters: {'num_layers': 1, 'hidden_size': 104, 'lstm_dropout': 0.1751789667861962, 'dense_dropout': 0.3677628987151054, 'learning_rate': 0.021420983611543182}. Best is trial 0 with value: 0.005818087141960859.
[I 2024-10-29 12:45:01,883] Trial 3 finished with value: 0.0017340044723823667 and parameters: {'num_layers': 1, 

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  69
  Number of complete trials:  31
Best trial:
  Value:  0.0015509831137023866
  Params: 
    num_layers: 2
    hidden_size: 64
    lstm_dropout: 0.2637510235281985
    dense_dropout: 0.039797036820399076
    learning_rate: 0.005842236882270005
R2 for sim at Tangnaihai(1972-2015) 0.9499199982410991


...

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  65
  Number of complete trials:  35
Best trial:
  Value:  0.002300985681358725
  Params: 
    num_layers: 1
    hidden_size: 24
    lstm_dropout: 0.13120629161417807
    dense_dropout: 0.09060595907531592
    learning_rate: 0.015132900391851245
R2 for sim at Xunhua(1972-2016) 0.9688751676698253


[I 2024-10-29 16:55:09,339] Trial 0 finished with value: 0.004443456535227597 and parameters: {'num_layers': 2, 'hidden_size': 104, 'lstm_dropout': 0.4304528192995433, 'dense_dropout': 0.2145202641673672, 'learning_rate': 0.04548005556683225}. Best is trial 0 with value: 0.004443456535227597.
[I 2024-10-29 16:56:05,028] Trial 1 finished with value: 0.014211467001587152 and parameters: {'num_layers': 3, 'hidden_size': 16, 'lstm_dropout': 0.26601314767425127, 'dense_dropout': 0.31776717578043806, 'learning_rate': 0.08182706243987062}. Best is trial 0 with value: 0.004443456535227597.
[I 2024-10-29 16:56:55,541] Trial 2 finished with value: 0.006931096315383911 and parameters: {'num_layers': 3, 'hidden_size': 120, 'lstm_dropout': 0.2650865405634053, 'dense_dropout': 0.4603634295140256, 'learning_rate': 0.000300929729989736}. Best is trial 0 with value: 0.004443456535227597.
[I 2024-10-29 16:57:49,220] Trial 3 finished with value: 0.008127463981509209 and parameters: {'num_layers': 3, 'hid

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  69
  Number of complete trials:  31
Best trial:
  Value:  0.0017146918107755482
  Params: 
    num_layers: 1
    hidden_size: 112
    lstm_dropout: 0.44909897977233487
    dense_dropout: 0.14687893693953827
    learning_rate: 0.012572113657889612
R2 for sim at Xunhua(1972-2017) 0.9538845882290177


[I 2024-10-29 17:18:14,247] Trial 0 finished with value: 0.004815324558876455 and parameters: {'num_layers': 2, 'hidden_size': 88, 'lstm_dropout': 0.2278988286119426, 'dense_dropout': 0.3450567889412119, 'learning_rate': 0.0476743657117066}. Best is trial 0 with value: 0.004815324558876455.
[I 2024-10-29 17:18:58,069] Trial 1 finished with value: 0.005404074210673571 and parameters: {'num_layers': 2, 'hidden_size': 80, 'lstm_dropout': 0.20267760106223393, 'dense_dropout': 0.013904744177716089, 'learning_rate': 5.173767723929824e-05}. Best is trial 0 with value: 0.004815324558876455.
[I 2024-10-29 17:19:48,609] Trial 2 finished with value: 0.005569370114244521 and parameters: {'num_layers': 3, 'hidden_size': 104, 'lstm_dropout': 0.2549482368858819, 'dense_dropout': 0.3144876668370799, 'learning_rate': 1.909282043858444e-05}. Best is trial 0 with value: 0.004815324558876455.
[I 2024-10-29 17:20:35,095] Trial 3 finished with value: 0.005210581934079528 and parameters: {'num_layers': 2, 'h

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  70
  Number of complete trials:  30
Best trial:
  Value:  0.0027120799059048295
  Params: 
    num_layers: 1
    hidden_size: 40
    lstm_dropout: 0.46060518971450587
    dense_dropout: 0.23992478496818537
    learning_rate: 0.0015863830382434499
R2 for sim at Xunhua(1972-2018) 0.8902277837139141
R2 for pred at Xunhua(2015-2019) -0.013836092512255593


In [30]:
selected_features = ['snomlt(mm)', 'eplant(mm)', 'surq_cha(mm)', 
                         'snofall(mm)', 'wet_oflo(mm)', 'wet_evap(mm)', 
                         'snopack(mm)', 'sw_change(mm)', 'MIN-TEM(C)','SWATPlusSimFlow','flow(m^3/s)']
start_year = 1972
pred_years = np.arange(2015,2020)
RETRAIN = True
name = 'InputOutputSamples_metesimyr_swatpsim'
batch_size = 32
ystar_col = "pred"
if not os.path.exists(sample_path):
    os.makedirs(sample_path)
for hydro_station in hydro_stations:
    sample_path = f'../samples/SequenceSamples/{hydro_station}/{name}/'
    if not os.path.exists(sample_path):
        os.makedirs(sample_path)
    result_path = f'../result/LSTMRegressorData/{hydro_station}/{name}/'
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    pred_df = pd.DataFrame()
    for pred_year in pred_years:
        model_path = f'../model/12MonthlyFlowProcessForecast_{hydro_station}/LSTMRegressorModel/{name}/pred{pred_year}/'
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        df = pd.read_csv(
            f'../samples/{name}/{hydro_station}_MeteAVGCalvalFeatureDataForML_PRED{pred_year}.csv',
            index_col=['date'], parse_dates=['date']
        )

        df = df[selected_features]

        # Check if 'ARIMASimFlow' exists in df columns
        if 'ARIMASimFlow' not in df.columns:
            print(f"Warning: ARIMASimFlow column not found in data for {hydro_station} station, year {pred_year}")
        
        target = 'flow(m^3/s)'
        features = list(df.columns.difference([target]))
        features = list(df.columns.copy())
        features.remove(target)
        cal = df.loc[:f'{pred_year-6}-12-31',:]
        test = df.loc[f'{pred_year-6}-01-01':f'{pred_year-1}-12-31',:]
        pre = df.loc[f'{pred_year-1}-01-01':f'{pred_year}-12-31',:]
        X_scaler = MinMaxScaler(feature_range=(0,1))
        Y_scaler = MinMaxScaler(feature_range=(0,1))
        X_scaler.fit(cal[features])
        Y_scaler.fit(cal[[target]])
        cal_X = X_scaler.transform(cal[features])
        test_X = X_scaler.transform(test[features])
        pre_X = X_scaler.transform(pre[features])
        cal_y = Y_scaler.transform(cal[[target]])
        test_y = Y_scaler.transform(test[[target]])
        pre_y = Y_scaler.transform(pre[[target]])
        cal = pd.concat([pd.DataFrame(cal_X,columns=features,index=cal.index),pd.DataFrame(cal_y,columns=[target],index=cal.index)],axis=1)
        test = pd.concat([pd.DataFrame(test_X,columns=features,index=test.index),pd.DataFrame(test_y,columns=[target],index=test.index)],axis=1)
        pre = pd.concat([pd.DataFrame(pre_X,columns=features,index=pre.index),pd.DataFrame(pre_y,columns=[target],index=pre.index)],axis=1)
        cal.to_csv(sample_path+f'cal_{start_year}_{pred_year-6}.csv')
        test.to_csv(sample_path+f'test_{pred_year-5}_{pred_year-1}.csv')
        pre.to_csv(sample_path+f'pre_{pred_year}.csv')

        sequence_length =12
        cal_dataset = SimulatedSequenceDataset(
            dataframe=cal.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        test_dataset = SimulatedSequenceDataset(
            dataframe=test.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )

        i=0
        X, y = cal_dataset[i]
        # print(X)
        # print(y)

        # print(cal_dataset.X.shape, cal_dataset.y.shape)
        # print(test_dataset.X.shape, test_dataset.y.shape)

        cal_index = cal.index[sequence_length:]
        test_index = test.index[sequence_length:]

        # print(len(cal_index)==cal_dataset.__len__())
        # print(len(test_index)==test_dataset.__len__())

       
        
        if not os.path.exists(model_path+'model.pickle'):
            objective = Objective(
                    train_dataset=cal_dataset,
                    val_dataset=test_dataset,
                    num_epoch=1000,# 1000
                    batch_size=batch_size,
                    shuffle=True,
                    model_path='./scheme/LSTMRegressor/',
            )
            study = optuna.create_study(
                    study_name='example-study',
                    direction='minimize',
                )
            study.optimize(objective, n_trials=100) #100

            pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
            complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])


            print("Study statistics: ")
            print("  Number of finished trials: ", len(study.trials))
            print("  Number of pruned trials: ", len(pruned_trials))
            print("  Number of complete trials: ", len(complete_trials))

            print("Best trial:")
            trial = study.best_trial

            save_best_trial(trial, model_path=model_path)

            save_model(trial, model_path=model_path)

            best_model_state = trial.user_attrs["best_model_state"]
            print("  Value: ", trial.value)
            print("  Params: ")
            for key, value in trial.params.items():
                print("    {}: {}".format(key, value))

            best_trial = load_best_trial(model_file=model_path+'best_trial.pickle')
            model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
            cal_loader = DataLoader(cal_dataset, batch_size=64, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

            cal_ = cal.loc[cal_index,[target]].copy()
            test_ = test.loc[test_index,[target]].copy()

            cal_[ystar_col] = predict(cal_loader, model).cpu().numpy()
            test_[ystar_col] = predict(test_loader, model).cpu().numpy()

            df_out = pd.concat((cal_, test_))[[target, ystar_col]]
            for c in df_out.columns:
                # df_out[c] = df_out[c] * target_std + target_mean
                df_out[c] = Y_scaler.inverse_transform(df_out[[c]])
            # print(df_out)
            df_out = df_out.rename(columns={target: 'obs'})
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            df_out.to_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv')
        else:
            df_out = pd.read_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv',index_col=['date'], parse_dates=['date'])
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            print(f'model already exists in {model_path}')

        # !Perform prediction
        pre = pd.read_csv(sample_path+f'pre_{pred_year}.csv',index_col=['date'], parse_dates=['date'])
        pre_dataset = SimulatedSequenceDataset(
            dataframe=pre.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        pre_loader = DataLoader(pre_dataset, batch_size=batch_size, shuffle=False)
        pre_index = pre.index[sequence_length:]
        pre_ = pre.loc[pre_index,[target]].copy()
        model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
        pre_[ystar_col] = predict(pre_loader, model).cpu().numpy()
        for c in pre_.columns:
            # df_out[c] = df_out[c] * target_std + target_mean
            pre_[c] = Y_scaler.inverse_transform(pre_[[c]])
        pred_df = pd.concat((pred_df, pre_), axis=0)
    pred_df = pred_df.rename(columns={'flow(m^3/s)': 'obs'})
    print(f'R2 for pred at {hydro_station}({pred_years[0]}-{pred_years[-1]})',r2_score(pred_df['obs'],pred_df['pred']))
    pred_df.to_csv(result_path+f'vif_lstm_pred_{pred_years[0]}_{pred_years[-1]}.csv')
    # Visualization of the flow(m^3/s) and VIF_LSTM of pred_df
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['obs'], mode='lines', name='obs'))
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['pred'], mode='lines', name='pred'))
    fig.show()


[I 2024-10-29 17:41:36,975] A new study created in memory with name: example-study


[I 2024-10-29 17:42:17,025] Trial 0 finished with value: 0.008319276617839932 and parameters: {'num_layers': 1, 'hidden_size': 128, 'lstm_dropout': 0.11967174195172825, 'dense_dropout': 0.25247167430523765, 'learning_rate': 0.0001894477963797847}. Best is trial 0 with value: 0.008319276617839932.
[I 2024-10-29 17:43:06,325] Trial 1 finished with value: 0.011516637168824673 and parameters: {'num_layers': 3, 'hidden_size': 32, 'lstm_dropout': 0.21537673519452372, 'dense_dropout': 0.3632978498483674, 'learning_rate': 0.016303129898019127}. Best is trial 0 with value: 0.008319276617839932.
[I 2024-10-29 17:43:50,573] Trial 2 finished with value: 0.014770777197554708 and parameters: {'num_layers': 2, 'hidden_size': 40, 'lstm_dropout': 0.26308433057057995, 'dense_dropout': 0.4151789716609489, 'learning_rate': 0.00010304838630040942}. Best is trial 0 with value: 0.008319276617839932.
[I 2024-10-29 17:44:30,490] Trial 3 finished with value: 0.011670557782053947 and parameters: {'num_layers': 1

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  75
  Number of complete trials:  25
Best trial:
  Value:  0.005909296218305826
  Params: 
    num_layers: 1
    hidden_size: 112
    lstm_dropout: 0.29559666891583236
    dense_dropout: 0.055581993446474375
    learning_rate: 0.007514444392600025
R2 for sim at Tangnaihai(1972-2014) 0.9471993783320701


[I 2024-10-29 17:59:08,541] Trial 0 finished with value: 0.01172504760324955 and parameters: {'num_layers': 1, 'hidden_size': 32, 'lstm_dropout': 0.2718539858561464, 'dense_dropout': 0.39414606662564733, 'learning_rate': 0.0010899639874682062}. Best is trial 0 with value: 0.01172504760324955.
[I 2024-10-29 17:59:49,334] Trial 1 finished with value: 0.01397743879351765 and parameters: {'num_layers': 1, 'hidden_size': 112, 'lstm_dropout': 0.4099643186166765, 'dense_dropout': 0.3350324450451822, 'learning_rate': 0.04029465374168289}. Best is trial 0 with value: 0.01172504760324955.
[I 2024-10-29 18:00:30,175] Trial 2 finished with value: 0.008026271825656295 and parameters: {'num_layers': 1, 'hidden_size': 56, 'lstm_dropout': 0.2698297516733577, 'dense_dropout': 0.25011877575027885, 'learning_rate': 0.00011334692669774551}. Best is trial 2 with value: 0.008026271825656295.
[I 2024-10-29 18:01:11,984] Trial 3 finished with value: 0.009168017073534429 and parameters: {'num_layers': 1, 'hidd

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  71
  Number of complete trials:  29
Best trial:
  Value:  0.006433308124542236
  Params: 
    num_layers: 1
    hidden_size: 80
    lstm_dropout: 0.4399119955876998
    dense_dropout: 0.0665443239953629
    learning_rate: 0.01336747758565618
R2 for sim at Tangnaihai(1972-2015) 0.9102051021248346


...

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  68
  Number of complete trials:  32
Best trial:
  Value:  0.004116126918233931
  Params: 
    num_layers: 1
    hidden_size: 88
    lstm_dropout: 0.024531865595348923
    dense_dropout: 0.092607411484683
    learning_rate: 0.008993789439421949
R2 for sim at Xunhua(1972-2016) 0.9369043230114699


[I 2024-10-29 21:05:50,201] Trial 0 finished with value: 0.002831778721883893 and parameters: {'num_layers': 3, 'hidden_size': 72, 'lstm_dropout': 0.27872715766925366, 'dense_dropout': 0.2952660428929327, 'learning_rate': 0.0003903213731982926}. Best is trial 0 with value: 0.002831778721883893.
[I 2024-10-29 21:06:38,530] Trial 1 finished with value: 0.0038843381917104125 and parameters: {'num_layers': 3, 'hidden_size': 80, 'lstm_dropout': 0.005771098782018647, 'dense_dropout': 0.050653639721585486, 'learning_rate': 1.0663979759307548e-05}. Best is trial 0 with value: 0.002831778721883893.
[I 2024-10-29 21:07:15,688] Trial 2 finished with value: 0.003982004011049867 and parameters: {'num_layers': 1, 'hidden_size': 104, 'lstm_dropout': 0.21290721949656072, 'dense_dropout': 0.21278188674969833, 'learning_rate': 0.02243074788710415}. Best is trial 0 with value: 0.002831778721883893.
[I 2024-10-29 21:07:53,865] Trial 3 finished with value: 0.007467061979696155 and parameters: {'num_layers'

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  73
  Number of complete trials:  27
Best trial:
  Value:  0.0016445262299384922
  Params: 
    num_layers: 1
    hidden_size: 32
    lstm_dropout: 0.297534118686633
    dense_dropout: 0.18698590305117418
    learning_rate: 0.009872952383076632
R2 for sim at Xunhua(1972-2017) 0.9308207286864563


[I 2024-10-29 21:25:05,920] Trial 0 finished with value: 0.009329840540885925 and parameters: {'num_layers': 1, 'hidden_size': 72, 'lstm_dropout': 0.39376371122983533, 'dense_dropout': 0.3524886869391458, 'learning_rate': 0.067457598380799}. Best is trial 0 with value: 0.009329840540885925.
[I 2024-10-29 21:25:51,230] Trial 1 finished with value: 0.006798525573685765 and parameters: {'num_layers': 1, 'hidden_size': 128, 'lstm_dropout': 0.37694207734439705, 'dense_dropout': 0.3697906936854813, 'learning_rate': 0.0011835119620884798}. Best is trial 1 with value: 0.006798525573685765.
[I 2024-10-29 21:26:46,210] Trial 2 finished with value: 0.002808980760164559 and parameters: {'num_layers': 2, 'hidden_size': 32, 'lstm_dropout': 0.05905645943743132, 'dense_dropout': 0.028150117063040325, 'learning_rate': 0.011895083716361925}. Best is trial 2 with value: 0.002808980760164559.
[I 2024-10-29 21:27:43,243] Trial 3 finished with value: 0.010359805542975664 and parameters: {'num_layers': 3, 'h

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  79
  Number of complete trials:  21
Best trial:
  Value:  0.0023393032024614513
  Params: 
    num_layers: 2
    hidden_size: 128
    lstm_dropout: 0.004797865588706263
    dense_dropout: 0.11829040400091506
    learning_rate: 0.00500105164235348
R2 for sim at Xunhua(1972-2018) 0.9560914987364005
R2 for pred at Xunhua(2015-2019) 0.3679475235466343


In [31]:
selected_features = ['P2020(mm)','MAX-TEM(C)','MIN-TEM(C)','flow(m^3/s)']
start_year = 1972
pred_years = np.arange(2015,2020)
RETRAIN = True
name = 'InputOutputSamples_metesimyr'
batch_size = 32
ystar_col = "pred"
if not os.path.exists(sample_path):
    os.makedirs(sample_path)
for hydro_station in hydro_stations:
    sample_path = f'../samples/SequenceSamples/{hydro_station}/{name}/'
    if not os.path.exists(sample_path):
        os.makedirs(sample_path)
    result_path = f'../result/LSTMRegressorData/{hydro_station}/{name}/'
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    pred_df = pd.DataFrame()
    for pred_year in pred_years:
        model_path = f'../model/12MonthlyFlowProcessForecast_{hydro_station}/LSTMRegressorModel/{name}/pred{pred_year}/'
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        df = pd.read_csv(
            f'../samples/{name}/{hydro_station}_MeteAVGCalvalFeatureDataForML_PRED{pred_year}.csv',
            index_col=['date'], parse_dates=['date']
        )

        df = df[selected_features]

        # Check if 'ARIMASimFlow' exists in df columns
        if 'ARIMASimFlow' not in df.columns:
            print(f"Warning: ARIMASimFlow column not found in data for {hydro_station} station, year {pred_year}")
        
        target = 'flow(m^3/s)'
        features = list(df.columns.difference([target]))
        features = list(df.columns.copy())
        features.remove(target)
        cal = df.loc[:f'{pred_year-6}-12-31',:]
        test = df.loc[f'{pred_year-6}-01-01':f'{pred_year-1}-12-31',:]
        pre = df.loc[f'{pred_year-1}-01-01':f'{pred_year}-12-31',:]
        X_scaler = MinMaxScaler(feature_range=(0,1))
        Y_scaler = MinMaxScaler(feature_range=(0,1))
        X_scaler.fit(cal[features])
        Y_scaler.fit(cal[[target]])
        cal_X = X_scaler.transform(cal[features])
        test_X = X_scaler.transform(test[features])
        pre_X = X_scaler.transform(pre[features])
        cal_y = Y_scaler.transform(cal[[target]])
        test_y = Y_scaler.transform(test[[target]])
        pre_y = Y_scaler.transform(pre[[target]])
        cal = pd.concat([pd.DataFrame(cal_X,columns=features,index=cal.index),pd.DataFrame(cal_y,columns=[target],index=cal.index)],axis=1)
        test = pd.concat([pd.DataFrame(test_X,columns=features,index=test.index),pd.DataFrame(test_y,columns=[target],index=test.index)],axis=1)
        pre = pd.concat([pd.DataFrame(pre_X,columns=features,index=pre.index),pd.DataFrame(pre_y,columns=[target],index=pre.index)],axis=1)
        cal.to_csv(sample_path+f'cal_{start_year}_{pred_year-6}.csv')
        test.to_csv(sample_path+f'test_{pred_year-5}_{pred_year-1}.csv')
        pre.to_csv(sample_path+f'pre_{pred_year}.csv')

        sequence_length =12
        cal_dataset = SimulatedSequenceDataset(
            dataframe=cal.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        test_dataset = SimulatedSequenceDataset(
            dataframe=test.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )

        i=0
        X, y = cal_dataset[i]
        # print(X)
        # print(y)

        # print(cal_dataset.X.shape, cal_dataset.y.shape)
        # print(test_dataset.X.shape, test_dataset.y.shape)

        cal_index = cal.index[sequence_length:]
        test_index = test.index[sequence_length:]

        # print(len(cal_index)==cal_dataset.__len__())
        # print(len(test_index)==test_dataset.__len__())

       
        
        if not os.path.exists(model_path+'model.pickle'):
            objective = Objective(
                    train_dataset=cal_dataset,
                    val_dataset=test_dataset,
                    num_epoch=1000,# 1000
                    batch_size=batch_size,
                    shuffle=True,
                    model_path='./scheme/LSTMRegressor/',
            )
            study = optuna.create_study(
                    study_name='example-study',
                    direction='minimize',
                )
            study.optimize(objective, n_trials=100) #100

            pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
            complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])


            print("Study statistics: ")
            print("  Number of finished trials: ", len(study.trials))
            print("  Number of pruned trials: ", len(pruned_trials))
            print("  Number of complete trials: ", len(complete_trials))

            print("Best trial:")
            trial = study.best_trial

            save_best_trial(trial, model_path=model_path)

            save_model(trial, model_path=model_path)

            best_model_state = trial.user_attrs["best_model_state"]
            print("  Value: ", trial.value)
            print("  Params: ")
            for key, value in trial.params.items():
                print("    {}: {}".format(key, value))

            best_trial = load_best_trial(model_file=model_path+'best_trial.pickle')
            model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
            cal_loader = DataLoader(cal_dataset, batch_size=64, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

            cal_ = cal.loc[cal_index,[target]].copy()
            test_ = test.loc[test_index,[target]].copy()

            cal_[ystar_col] = predict(cal_loader, model).cpu().numpy()
            test_[ystar_col] = predict(test_loader, model).cpu().numpy()

            df_out = pd.concat((cal_, test_))[[target, ystar_col]]
            for c in df_out.columns:
                # df_out[c] = df_out[c] * target_std + target_mean
                df_out[c] = Y_scaler.inverse_transform(df_out[[c]])
            # print(df_out)
            df_out = df_out.rename(columns={target: 'obs'})
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            df_out.to_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv')
        else:
            df_out = pd.read_csv(result_path+f'cal_test_sim_{start_year}_{pred_year-1}.csv',index_col=['date'], parse_dates=['date'])
            print(f'R2 for sim at {hydro_station}({start_year}-{pred_year-1})',r2_score(df_out['obs'],df_out[ystar_col]))
            print(f'model already exists in {model_path}')

        # !Perform prediction
        pre = pd.read_csv(sample_path+f'pre_{pred_year}.csv',index_col=['date'], parse_dates=['date'])
        pre_dataset = SimulatedSequenceDataset(
            dataframe=pre.copy(),
            target=target,
            features=features,
            sequence_length=sequence_length
        )
        pre_loader = DataLoader(pre_dataset, batch_size=batch_size, shuffle=False)
        pre_index = pre.index[sequence_length:]
        pre_ = pre.loc[pre_index,[target]].copy()
        model = load_model(model_file=model_path+'model.pickle').to(DEVICE)
        pre_[ystar_col] = predict(pre_loader, model).cpu().numpy()
        for c in pre_.columns:
            # df_out[c] = df_out[c] * target_std + target_mean
            pre_[c] = Y_scaler.inverse_transform(pre_[[c]])
        pred_df = pd.concat((pred_df, pre_), axis=0)
    pred_df = pred_df.rename(columns={'flow(m^3/s)': 'obs'})
    print(f'R2 for pred at {hydro_station}({pred_years[0]}-{pred_years[-1]})',r2_score(pred_df['obs'],pred_df['pred']))
    pred_df.to_csv(result_path+f'vif_lstm_pred_{pred_years[0]}_{pred_years[-1]}.csv')
    # Visualization of the flow(m^3/s) and VIF_LSTM of pred_df
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['obs'], mode='lines', name='obs'))
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['pred'], mode='lines', name='pred'))
    fig.show()

[I 2024-10-29 21:40:53,090] A new study created in memory with name: example-study


[I 2024-10-29 21:41:34,438] Trial 0 finished with value: 0.0029427498811855912 and parameters: {'num_layers': 3, 'hidden_size': 104, 'lstm_dropout': 0.1282230949305253, 'dense_dropout': 0.010771193831729309, 'learning_rate': 0.01365923970744934}. Best is trial 0 with value: 0.0029427498811855912.
[I 2024-10-29 21:42:16,798] Trial 1 finished with value: 0.013283176813274622 and parameters: {'num_layers': 3, 'hidden_size': 32, 'lstm_dropout': 0.479898610523583, 'dense_dropout': 0.266965389228919, 'learning_rate': 1.839001818641712e-05}. Best is trial 0 with value: 0.0029427498811855912.
[I 2024-10-29 21:43:00,860] Trial 2 finished with value: 0.00395623967051506 and parameters: {'num_layers': 3, 'hidden_size': 88, 'lstm_dropout': 0.1548382453141704, 'dense_dropout': 0.21955736376697027, 'learning_rate': 0.013415285606111023}. Best is trial 0 with value: 0.0029427498811855912.
[I 2024-10-29 21:43:34,926] Trial 3 finished with value: 0.007723681163042784 and parameters: {'num_layers': 1, '

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  75
  Number of complete trials:  25
Best trial:
  Value:  0.0019671354675665498
  Params: 
    num_layers: 2
    hidden_size: 72
    lstm_dropout: 0.14044621251482972
    dense_dropout: 0.05275544516766682
    learning_rate: 0.015476881437324523
R2 for sim at Tangnaihai(1972-2014) 0.9605971176455003


[I 2024-10-29 21:56:27,634] Trial 0 finished with value: 0.0036265439121052623 and parameters: {'num_layers': 1, 'hidden_size': 112, 'lstm_dropout': 0.3869414340554768, 'dense_dropout': 0.11454429311534758, 'learning_rate': 0.001028949843289967}. Best is trial 0 with value: 0.0036265439121052623.
[I 2024-10-29 21:57:19,564] Trial 1 finished with value: 0.11066295951604843 and parameters: {'num_layers': 3, 'hidden_size': 64, 'lstm_dropout': 0.16611703613439688, 'dense_dropout': 0.3685412746689556, 'learning_rate': 0.07926088725229255}. Best is trial 0 with value: 0.0036265439121052623.
[I 2024-10-29 21:58:09,886] Trial 2 finished with value: 0.005450853379443288 and parameters: {'num_layers': 3, 'hidden_size': 128, 'lstm_dropout': 0.25661614947998873, 'dense_dropout': 0.3229598600892425, 'learning_rate': 0.01078214106169636}. Best is trial 0 with value: 0.0036265439121052623.
[I 2024-10-29 21:58:50,886] Trial 3 finished with value: 0.0036571924574673176 and parameters: {'num_layers': 1,

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  76
  Number of complete trials:  24
Best trial:
  Value:  0.0016205926658585668
  Params: 
    num_layers: 2
    hidden_size: 64
    lstm_dropout: 0.0756546407129671
    dense_dropout: 0.028002001212297664
    learning_rate: 0.007327891968750374
R2 for sim at Tangnaihai(1972-2015) 0.9042078095657861


...

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  63
  Number of complete trials:  37
Best trial:
  Value:  0.0016664667637087405
  Params: 
    num_layers: 1
    hidden_size: 128
    lstm_dropout: 0.1751526390038343
    dense_dropout: 0.07014923951027559
    learning_rate: 0.018187861352936367
R2 for sim at Xunhua(1972-2016) 0.9708941843686503


[I 2024-10-30 00:50:11,612] Trial 0 finished with value: 0.00376211223192513 and parameters: {'num_layers': 1, 'hidden_size': 32, 'lstm_dropout': 0.36442384345226525, 'dense_dropout': 0.05713672153974031, 'learning_rate': 6.255224480150893e-05}. Best is trial 0 with value: 0.00376211223192513.
[I 2024-10-30 00:50:44,361] Trial 1 finished with value: 0.003265012288466096 and parameters: {'num_layers': 1, 'hidden_size': 56, 'lstm_dropout': 0.10648842672945968, 'dense_dropout': 0.19677539553916812, 'learning_rate': 8.356618131931402e-05}. Best is trial 1 with value: 0.003265012288466096.
[I 2024-10-30 00:51:22,623] Trial 2 finished with value: 0.0024285426479764283 and parameters: {'num_layers': 3, 'hidden_size': 104, 'lstm_dropout': 0.4086900580756615, 'dense_dropout': 0.1517723088204047, 'learning_rate': 7.617361486447501e-05}. Best is trial 2 with value: 0.0024285426479764283.
[I 2024-10-30 00:51:54,613] Trial 3 finished with value: 0.003245683154091239 and parameters: {'num_layers': 1

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  57
  Number of complete trials:  43
Best trial:
  Value:  0.001650667458306998
  Params: 
    num_layers: 2
    hidden_size: 72
    lstm_dropout: 0.032399367231173176
    dense_dropout: 0.12860833882048817
    learning_rate: 0.007965345776222715
R2 for sim at Xunhua(1972-2017) 0.9526204865089549


[I 2024-10-30 01:16:29,368] Trial 0 finished with value: 0.003747583949007094 and parameters: {'num_layers': 3, 'hidden_size': 80, 'lstm_dropout': 0.37964189057182895, 'dense_dropout': 0.2442765563514594, 'learning_rate': 0.00021843807279510306}. Best is trial 0 with value: 0.003747583949007094.
[I 2024-10-30 01:17:12,465] Trial 1 finished with value: 0.005353954387828708 and parameters: {'num_layers': 3, 'hidden_size': 8, 'lstm_dropout': 0.3809353769489302, 'dense_dropout': 0.04322386628640523, 'learning_rate': 0.0668430382733256}. Best is trial 0 with value: 0.003747583949007094.
[I 2024-10-30 01:17:46,750] Trial 2 finished with value: 0.009128424571827054 and parameters: {'num_layers': 1, 'hidden_size': 72, 'lstm_dropout': 0.22838095608471365, 'dense_dropout': 0.48932519515732886, 'learning_rate': 0.0010045778213617932}. Best is trial 0 with value: 0.003747583949007094.
[I 2024-10-30 01:18:31,458] Trial 3 finished with value: 0.006089309579692781 and parameters: {'num_layers': 3, 'h

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  76
  Number of complete trials:  24
Best trial:
  Value:  0.0027137468568980694
  Params: 
    num_layers: 2
    hidden_size: 88
    lstm_dropout: 0.18130728360354695
    dense_dropout: 0.13452296363045613
    learning_rate: 0.004754276830086625
R2 for sim at Xunhua(1972-2018) 0.953830138844613
R2 for pred at Xunhua(2015-2019) 0.3561776205338002
